# Rauk's table and Wolfsberrg-Helmholz approximation

So far the Hamiltonians treated by the documentation have been focused in carbon chains. With this new update it is possible to work with different atoms, using the Rauk's table or the distance beetween the atoms with the Wolfsberrg-Helmholz aproximation.

To generate a Hamiltonian with atoms different from carbon, simply apply the same logic as before: Define a list of tuples with the atoms indexed by the position of their respective site in the compound. The new module of the library has two main purposes:

1) If you define the system using atoms different from carbons and the connectivity as ``integer`` numbers, the program will assign to the one body term the $\alpha$ and $\beta$ precomputed based on the Rauk's table.

2) If you define the system using atoms different from carbons and the connectivity as `float` numbers, the program will assign to the one body term the $\alpha$ and $\beta$ computed using the Wolfsberrg-Helmholz approximation, based on the provided distance.



## Example: Rauk's table

Suppose that you want to define a Hubbard model assigning the values based on the Rauk's table (RAUK, 2001). The calculations of $\alpha$ and $\beta$ were made based on Simple Hückel Molecular Orbital Theory(SHMO), following the following equations, for the atoms X, Y:

$$
\alpha_X = \alpha h_x|\beta| 
$$
$$
b_{XY} =  \kappa_{XY}|\beta|
$$

‌

You can do this based on the example below. 




In [1]:
import sys  
sys.path.insert(0, '../')
import numpy as np
from moha import HamHub


system = [('C1', 'Cl2', 1), ('Cl2', 'F3', 1),
                    ('F3', 'Si4', 1), ('Si4', 'C1', 1)]
hubbard = HamHub(system, u_onsite=np.array([1, 1]))

print(hubbard.generate_one_body_integral(dense=True, basis='spatial basis'))

[[-0.414    -0.033046  0.       -0.039975]
 [-0.033046 -0.492884 -0.027183  0.      ]
 [ 0.       -0.027183 -0.558443 -0.009061]
 [-0.039975  0.       -0.009061 -0.414   ]]


## Example:  Wolfsberrg-Helmholz approximation

Suppose now that you want to define the same system, but computing the parameters based on the overlap distance beetween the neighbors atoms. You can do that using the example below.

The values of $\alpha$ and $\beta$ are computed based on the following formulas:


$$
\alpha_X = -(\text{ionization potential of atom X})
$$

The $\beta$ value for the interaction between atom X and atom Y is defined as:

$$
\beta_{XY} = 1.75 S_{XY} \frac{\alpha_X + \alpha_Y}{2}
$$

In [2]:
import sys  
sys.path.insert(0, '../')
import numpy as np
from moha import HamHub


system = [('C1', 'Cl2', 1.5, 'pi'), ('Cl2', 'F3', 1.8, 'sigma'),
                    ('F3', 'Si4', 1.8, 'sigma'), ('Si4', 'C1', 1.7, 'sigma')]
hubbard = HamHub(system, u_onsite=np.array([1, 1]))

print(hubbard.generate_one_body_integral(dense=True, basis='spatial basis'))

[[-0.41380839 -0.73648465  0.         -0.56689357]
 [-0.73648465 -0.47655051 -0.81351185  0.        ]
 [ 0.         -0.81351185 -0.64027609 -0.60447297]
 [-0.56689357  0.         -0.60447297 -0.29956945]]


# Example: Usage with molecules 

If you want to make a chain using molecules you can use the following structure: pass the atom $X_n$ in the position m, the tuple would have the following form `Xm(n)`. Below it's possible to see a example of this. 

In [3]:
from moha import HamHuck


N2_O1_P2 = HamHuck([('N1(2)', 'O2(1)', 1), ('O2(1)', 'P3(2)', 1), ('P3(2)', 'N1(2)', 1)])
h1 = N2_O1_P2.generate_one_body_integral(dense=True, basis='spatial basis')
print(h1)

[[-0.441183 -0.060762 -0.041574]
 [-0.060762 -0.465701 -0.039975]
 [-0.041574 -0.039975 -0.424127]]


## Example: Defining the dictionaries 

It is also possible to define your own dictionary, basically pass the dictionaries following the structure:

1) `atom_dictionary = {'atom1': alpha1, 'atom2': alpha2}`
2) `bond_dictionary = {'atom1atom2': beta}`

In [4]:
import sys  
sys.path.insert(0, '../')
import numpy as np
from moha import HamHub

# First way to define the Hamiltonian
# two site Hubbard model

# system = [('C1', 'C2', 1)] is a list of tuples, where each tuple represents a bond
# between two atoms and the third element is the type of bond (singe or double).
# For now, we only support single bonds between carbon atoms. 
# For this type of bonds the default values of alpha and beta are -0.414 and -0.0533, respectively.
# In the future we are planning to support different types of bonds for different atoms.
system = [('C1', 'Cl2', 1), ('Cl2', 'F3', 1),
                    ('F3', 'Si4', 1), ('Si4', 'C1', 1)]
hubbard = HamHub(system, u_onsite=np.array([1, 1]), 
                 atom_dictionary={'C': 1, 'Cl': 2, 'F': 3, 'Si': 4},
                 bond_dictionary={'C,Cl': 0, 'Cl,F': 1, 'F,Si': 2, 'Si,C': 3})

print(hubbard.generate_one_body_integral(dense=True, basis='spatial basis'))

[[1. 0. 0. 0.]
 [0. 2. 1. 0.]
 [0. 0. 3. 2.]
 [3. 0. 0. 4.]]


In [5]:
import sys  
sys.path.insert(0, '../')
import numpy as np
from moha import HamHub

# First way to define the Hamiltonian
# two site Hubbard model

# system = [('C1', 'C2', 1)] is a list of tuples, where each tuple represents a bond
# between two atoms and the third element is the type of bond (singe or double).
# For now, we only support single bonds between carbon atoms. 
# For this type of bonds the default values of alpha and beta are -0.414 and -0.0533, respectively.
# In the future we are planning to support different types of bonds for different atoms.
system = [('C1', 'Cl2', 1.1), ('Cl2', 'F3', 1.0),
                    ('F3', 'Si4', 1.0), ('Si4', 'C1', 1.0)]
hubbard = HamHub(system, u_onsite=np.array([1, 1]),
                 orbital_overlap=np.array([[0, 1,0,0],[0, 0,3,0], [0, 0,0,4], [0, 0,0,1]]))

print(hubbard.generate_one_body_integral(dense=True, basis='spatial basis'))

[[-0.41380839 -0.77906404  0.          0.        ]
 [-0.77906404 -0.47655051 -2.93166983  0.        ]
 [ 0.         -2.93166983 -0.64027609 -3.28945939]
 [ 0.          0.         -3.28945939 -0.29956945]]


## Example: Second Body Terms: PariserParr Module

In [6]:
# Example: generating XXZ Heisenberg model 
# Returning electron integrals in a spatial orbital basis
# Assuming 4-fold symmetry
# Returning output as dense matrix

# In the future we are planning to support different types of bonds for different atoms.
system = [('C1', 'Cl2', 1.1), ('Cl2', 'F3', 1.0),
                    ('F3', 'Si4', 1.0), ('Si4', 'C1', 1.0)]
ham = HamHub(system, u_onsite=np.array([1, 1,1,1]),
                 orbital_overlap=np.array([[0, 1, 0, 0],[0, 0,3,0], [0, 0,0,4], [0, 0,0,1]]))

e0 = ham.generate_zero_body_integral()
h1 = ham.generate_one_body_integral(dense=True, basis='spatial basis') 
h2 = ham.generate_two_body_integral(dense=True, basis='spatial basis', sym=4)

print("Zero energy: ", e0)
print("One body integrals in spatial basis: \n", h1)
print("Shape of two body integral in spatial basis: ", h2.shape)
print("-"*60)

# Example: generating XXZ Heisenberg model in spin orbital basis
# Assuming 4-fold symmetry
# Returning output as dense matrix

h1 = ham.generate_one_body_integral(dense=True, basis='spinorbital basis')
h2 = ham.generate_two_body_integral(dense=True, basis='spinorbital basis', sym=4)

print("One body integrals in spin basis: \n", h1)
print("Shape of two body integral in spinorbital basis: ", h2.shape)

Zero energy:  0
One body integrals in spatial basis: 
 [[-0.41380839 -0.77906404  0.          0.        ]
 [-0.77906404 -0.47655051 -2.93166983  0.        ]
 [ 0.         -2.93166983 -0.64027609 -3.28945939]
 [ 0.          0.         -3.28945939 -0.29956945]]
Shape of two body integral in spatial basis:  (4, 4, 4, 4)
------------------------------------------------------------
One body integrals in spin basis: 
 [[-0.41380839 -0.77906404  0.          0.          0.          0.
   0.          0.        ]
 [-0.77906404 -0.47655051 -2.93166983  0.          0.          0.
   0.          0.        ]
 [ 0.         -2.93166983 -0.64027609 -3.28945939  0.          0.
   0.          0.        ]
 [ 0.          0.         -3.28945939 -0.29956945  0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.         -0.41380839 -0.77906404
   0.          0.        ]
 [ 0.          0.          0.          0.         -0.77906404 -0.47655051
  -2.93166983  0.        ]
 [ 0.     

In [7]:
# Example: generating 6 site Hubbard model 
# Returning electron integrals in a spatial orbital basis
# Assuming 8-fold symmetry
# Returning output as dense matrix

connectivity = np.array([[0, 1, 0, 0, 0, 1],
                         [1, 0, 1, 0, 0, 0],
                         [0, 1, 0, 1, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [0, 0, 0, 1, 0, 1],
                         [1, 0, 0, 0, 1, 0]])
hubbard = HamHub(connectivity,
                 alpha=0, 
                 beta=-1, 
                 u_onsite=np.array([1, 1, 1, 1, 1, 1]))

e0 = hubbard.generate_zero_body_integral()
h1 = hubbard.generate_one_body_integral(dense=True, basis='spatial basis') 
h2 = hubbard.generate_two_body_integral(dense=True, basis='spatial basis', sym=8)

print("Zero energy: ", e0)
print("One body integrals in spatial basis: \n", h1)
print("Shape of two body integral in spatial basis: ", h2.shape)
print("-"*60)

# Example: generating Hubbard model in spin orbital basis
# Assuming 8-fold symmetry
# Returning output as dense matrix

h1 = hubbard.generate_one_body_integral(dense=True, basis='spinorbital basis')
h2 = hubbard.generate_two_body_integral(dense=True, basis='spinorbital basis', sym=4)

print("One body integrals in spin basis: \n", h1)
print("Shape of two body integral in spinorbital basis: ", h2.shape)

Zero energy:  0
One body integrals in spatial basis: 
 [[ 0. -1.  0.  0.  0. -1.]
 [-1.  0. -1.  0.  0.  0.]
 [ 0. -1.  0. -1.  0.  0.]
 [ 0.  0. -1.  0. -1.  0.]
 [ 0.  0.  0. -1.  0. -1.]
 [-1.  0.  0.  0. -1.  0.]]
Shape of two body integral in spatial basis:  (6, 6, 6, 6)
------------------------------------------------------------
One body integrals in spin basis: 
 [[ 0. -1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [-1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0. -1.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0. -1.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0. -1.  0.]]
Shape of two 

## References:

[1]A. Orbital Interaction Theory of Organic Chemistry. [s.l.] John Wiley & Sons, 2004.

[2] https://en.wikipedia.org/wiki/Electron_affinity_(data_page)